In [4]:
# parent_folder_path = "/Users/haleyoro/Desktop/" # work on library computer
parent_folder_path = "/Users/Haley/Desktop/Neuroscience Lab/Repository/" # work on local computer

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pyabf

Load `sheets` dict from all_functions

In [12]:
cell_types_df = pd.read_csv(parent_folder_path+"murray-neuroscience-lab/Excel processor/List of cells.csv")
cell_types_df = cell_types_df.dropna(how='all')
cell_types_df = cell_types_df.iloc[:,:4]
cell_types_df.reset_index(drop=True,inplace=True)
sheets_types = cell_types_df["Cell"].unique().tolist()
print("Number of labeled sheets:",len(sheets_types))
sheet_names_df = pd.read_csv(parent_folder_path+'murray-neuroscience-lab/Processed CSVs/sheet_names.csv', header=None)
sheet_names = sheet_names_df.iloc[:,0].values
sheets = {}

for sheet in sheet_names:
    file_path = parent_folder_path+"murray-neuroscience-lab/Processed CSVs/"+sheet+".csv"
    df = pd.read_csv(file_path)
    df[["Trace name","Tags","Type"]] = df[["Trace name","Tags","Type"]].astype("string")
    if sheet in sheets_types:
        types = cell_types_df[cell_types_df["Cell"]==sheet]
        # df["Cell Type"] = types.iloc[0][1]
        # df["Input Resistance"] = types.iloc[0][2]
        # df["Motoneuron"] = types.iloc[0][3]
        if not df.empty:
            df.loc[:,"Cell Type"] = types.iloc[0]["Cell Type"]
            df.loc[:,"Input Resistance"] = types.iloc[0]["Input Resistance"]
            df.loc[:,"Motoneuron"] = types.iloc[0]["Motoneuron"]
    sheets[sheet] = df

sheets

Number of labeled sheets: 29


{'2012_04_25_cell1':           Trace name  ID  On time     Freq               Tags        Type  \
 0    2012_04_25_0004   1  1285.99      NaN  bout start; shock  Inhibitory   
 1    2012_04_25_0004   2  1305.92  50.1756               <NA>  Inhibitory   
 2    2012_04_25_0004   3  1340.95  28.5470               <NA>  Inhibitory   
 3    2012_04_25_0004   4  1371.30  32.9489               <NA>  Inhibitory   
 4    2012_04_25_0004   5  1405.23  29.4724           bout end  Inhibitory   
 ..               ...  ..      ...      ...                ...         ...   
 153  2012_04_25_0002  13  4726.57  39.7141               <NA>  Excitatory   
 154  2012_04_25_0002  14  4755.30  34.8068               <NA>  Excitatory   
 155  2012_04_25_0002  15  4780.80  39.2157               <NA>  Excitatory   
 156  2012_04_25_0002  16  4808.89  35.5999               <NA>  Excitatory   
 157  2012_04_25_0002  17  4846.59  26.5252           bout end  Excitatory   
 
      Seconds Cell Type  Input Resistance 

In [13]:
parent_folder_path_ABFS = "/Users/Haley/Desktop/ABF files annotated/"
for sheet in sheets.keys():
    df = sheets[sheet]
    traces = df["Trace name"].unique().tolist() # creates an array of the trace names annotated in the sheet
    # load the abf files for each sheet into a dictionary 
    abfs = {}
    for trace in traces:
        abfs[trace] = []
        if trace == "2012_03_21_0027":
            continue
        else:
            file_path2 = parent_folder_path_ABFS + trace +".abf"
            abf = pyabf.ABF(file_path2)
            abfs[trace]= abf
    # adds the dictionary to sheets 
    sheets[sheet] = {
        "annotations": sheets[sheet],
        "abfs": abfs
    }

In [14]:
abf = sheets['2013_03_21_cell5']["abfs"]['2013_03_21_0049']
df_full = sheets['2013_03_21_cell5']["annotations"]
df = df_full[df_full["Trace name"]=="2013_03_21_0049"]

In [23]:
cell_types_df = pd.read_csv(parent_folder_path+"murray-neuroscience-lab/Excel processor/List of cells.csv")
cell_types_df = cell_types_df.dropna(how='all')
cell_types_df = cell_types_df.iloc[:,:4]
cell_types_df.reset_index(drop=True,inplace=True)

types = cell_types_df[cell_types_df["Cell"]=="2012_04_25_cell1"]
df.loc[:,"Cell Type"] = types.iloc[0]["Cell Type"]
df.loc[:,"Input Resistance"] = types.iloc[0]["Input Resistance"]
df.loc[:,"Motoneuron"] = types.iloc[0]["Motoneuron"]

In [15]:
abf.setSweep(sweepNumber=0, channel=0)
x = abf.sweepX
y = abf.sweepY
abf_df = pd.DataFrame({
    'Time': x,
    'Current': y
})
print("abf_df head:", abf_df.head())  # Check if abf_df is populated correctly

# df["On time"] = pd.to_numeric(df["On time"], errors="coerce")
# df["Seconds"] = df['On time']*0.001
waveforms = {}
for i in range(len(df) - 1):
    t_0 = df.iloc[i,6] 
    t_f = df.iloc[i+1,6]

    # phase_0 = int(t_0 * 100000)
    # phase_1 = int(t_f * 100000)
    # abf_waveform = abf_df.loc[phase_0:phase_1].copy()
    abf_waveform = abf_df[(abf_df["Time"] >= t_0) & (abf_df["Time"] <= t_f)].copy()
    print(f"abf_waveform shape (i={i}):", abf_waveform.shape) 
    # print(abf_waveform)
    phase = (abf_waveform["Time"] - t_0) / (t_f - t_0)
    abf_waveform["Phase"] = phase
    
    # NORMALIZE CURRENT ALSO, test this:
    
    # y_max = max(abf_waveform["Current"])
    # y_min = min(abf_waveform["Current"])
    # normalized = (abf_waveform["Current"] - y_min) / (y_max - y_min)
    # abf_waveform["Normalized Current"] = normalized
    #=====

    # Dict keys
    freq = 1 / (t_f-t_0) # CHECK = df.loc[i + 1, "Freq"]
    cell_type = df.iloc[0,5]
    signal_type = df.iloc[0,7]
    fast_slow = df.iloc[0,9] # 8:resistance 9:prim/sec low/high rin need to see what is fast/slow
    key = (freq,cell_type,signal_type,fast_slow)
    waveforms[key] = abf_waveform



abf_df head:       Time    Current
0  0.00000 -13.183593
1  0.00001 -14.770507
2  0.00002 -15.869140
3  0.00003 -15.319823
4  0.00004 -13.793944
abf_waveform shape (i=0): (0, 2)
abf_waveform shape (i=1): (0, 2)
abf_waveform shape (i=2): (0, 2)
abf_waveform shape (i=3): (0, 2)
abf_waveform shape (i=4): (0, 2)
abf_waveform shape (i=5): (0, 2)
abf_waveform shape (i=6): (0, 2)
abf_waveform shape (i=7): (0, 2)
abf_waveform shape (i=8): (0, 2)
abf_waveform shape (i=9): (0, 2)
abf_waveform shape (i=10): (0, 2)
abf_waveform shape (i=11): (0, 2)
abf_waveform shape (i=12): (0, 2)
abf_waveform shape (i=13): (0, 2)
abf_waveform shape (i=14): (0, 2)
abf_waveform shape (i=15): (0, 2)
abf_waveform shape (i=16): (0, 2)
abf_waveform shape (i=17): (0, 2)
abf_waveform shape (i=18): (0, 2)
abf_waveform shape (i=19): (0, 2)
abf_waveform shape (i=20): (0, 2)
abf_waveform shape (i=21): (0, 2)
abf_waveform shape (i=22): (0, 2)
abf_waveform shape (i=23): (0, 2)
abf_waveform shape (i=24): (0, 2)


In [16]:
waveforms

{(np.float64(25.233409033558843),
  'Excitatory',
  'CaP',
  'primary'): Empty DataFrame
 Columns: [Time, Current, Phase]
 Index: [],
 (np.float64(32.61578604044627),
  'Excitatory',
  'CaP',
  'primary'): Empty DataFrame
 Columns: [Time, Current, Phase]
 Index: [],
 (np.float64(36.50967506389131),
  'Excitatory',
  'CaP',
  'primary'): Empty DataFrame
 Columns: [Time, Current, Phase]
 Index: [],
 (np.float64(26.212319790300285),
  'Excitatory',
  'CaP',
  'primary'): Empty DataFrame
 Columns: [Time, Current, Phase]
 Index: [],
 (np.float64(22.836263987212853),
  'Excitatory',
  'CaP',
  'primary'): Empty DataFrame
 Columns: [Time, Current, Phase]
 Index: [],
 (np.float64(27.21829069134327),
  'Excitatory',
  'CaP',
  'primary'): Empty DataFrame
 Columns: [Time, Current, Phase]
 Index: [],
 (np.float64(28.137310073156204),
  'Excitatory',
  'CaP',
  'primary'): Empty DataFrame
 Columns: [Time, Current, Phase]
 Index: [],
 (np.float64(1.0798319781442016),
  'Excitatory',
  'CaP',
  'pri

In [29]:
df

,Trace name,ID,On time,Freq,Tags,Type,Seconds,Cell Type,Input Resistance,Motoneuron
0,2013_03_21_0049,1,21148.96,NaN,bout start; shock,Excitatory,21.14896,CaP,33.0,primary
1,2013_03_21_0049,2,21188.59,25.2334,<NA>,Excitatory,21.18859,CaP,33.0,primary
2,2013_03_21_0049,3,21219.25,32.6158,<NA>,Excitatory,21.21925,CaP,33.0,primary
3,2013_03_21_0049,4,21246.64,36.5097,<NA>,Excitatory,21.24664,CaP,33.0,primary
4,2013_03_21_0049,5,21284.79,26.2123,<NA>,Excitatory,21.28479,CaP,33.0,primary
5,2013_03_21_0049,6,21328.58,22.8363,<NA>,Excitatory,21.32858,CaP,33.0,primary
6,2013_03_21_0049,7,21365.32,27.2183,<NA>,Excitatory,21.36532,CaP,33.0,primary
7,2013_03_21_0049,8,21400.86,28.1373,bout end,Excitatory,21.40086,CaP,33.0,primary
8,2013_03_21_0049,9,22326.93,1.0798,bout start; shock,Excitatory,22.32693,CaP,33.0,primary
9,2013_03_21_0049,10,22351.45,40.7830,<NA>,Excitatory,22.35145,CaP,33.0,primary


In [28]:
print(abf_df.head())
print(abf_df.shape)
print(abf_df["Time"].min(), abf_df["Time"].max())


      Time    Current
0  0.00000 -13.183593
1  0.00001 -14.770507
2  0.00002 -15.869140
3  0.00003 -15.319823
4  0.00004 -13.793944
(1000000, 2)
0.0 9.99999


In [30]:
df

,Trace name,ID,On time,Freq,Tags,Type,Seconds,Cell Type,Input Resistance,Motoneuron
0,2013_03_21_0049,1,21148.96,NaN,bout start; shock,Excitatory,21.14896,CaP,33.0,primary
1,2013_03_21_0049,2,21188.59,25.2334,<NA>,Excitatory,21.18859,CaP,33.0,primary
2,2013_03_21_0049,3,21219.25,32.6158,<NA>,Excitatory,21.21925,CaP,33.0,primary
3,2013_03_21_0049,4,21246.64,36.5097,<NA>,Excitatory,21.24664,CaP,33.0,primary
4,2013_03_21_0049,5,21284.79,26.2123,<NA>,Excitatory,21.28479,CaP,33.0,primary
5,2013_03_21_0049,6,21328.58,22.8363,<NA>,Excitatory,21.32858,CaP,33.0,primary
6,2013_03_21_0049,7,21365.32,27.2183,<NA>,Excitatory,21.36532,CaP,33.0,primary
7,2013_03_21_0049,8,21400.86,28.1373,bout end,Excitatory,21.40086,CaP,33.0,primary
8,2013_03_21_0049,9,22326.93,1.0798,bout start; shock,Excitatory,22.32693,CaP,33.0,primary
9,2013_03_21_0049,10,22351.45,40.7830,<NA>,Excitatory,22.35145,CaP,33.0,primary


In [27]:
abf.setSweep(sweepNumber=0, channel=0)
x = abf.sweepX
y = abf.sweepY
abf_df = pd.DataFrame({
    'Time': x,
    'Current': y
})
# print("abf_df head:", abf_df.head())  # Check if abf_df is populated correctly

# df["On time"] = pd.to_numeric(df["On time"], errors="coerce")
df["Seconds"] = df['On time']*0.001
waveforms = {}
for i in range(len(df) - 1):
    t_0 = df.iloc[i]["Seconds"]
    t_f = df.iloc[i+1]["Seconds"]
    print(t_0,t_f)
    # phase_0 = int(t_0 * 100000)
    # phase_1 = int(t_f * 100000)
    # abf_waveform = abf_df.loc[phase_0:phase_1].copy()
    abf_waveform = abf_df[(abf_df["Time"] >= t_0) & (abf_df["Time"] <= t_f)].copy()
    # print(f"abf_waveform shape (i={i}):", abf_waveform.shape) 
    print(abf_waveform)
    phase = (abf_waveform["Time"] - t_0) / (t_f - t_0)
    abf_waveform["Phase"] = phase
    
    # NORMALIZE CURRENT ALSO, test this:
    
    # y_max = max(abf_waveform["Current"])
    # y_min = min(abf_waveform["Current"])
    # normalized = (abf_waveform["Current"] - y_min) / (y_max - y_min)
    # abf_waveform["Normalized Current"] = normalized
    #=====

    # Dict keys
    freq = 1 / (t_f-t_0) # CHECK = df.loc[i + 1, "Freq"]
    signal_type = df.iloc[0]["Type"]
    cell_type = df.iloc[0]["Cell Type"]
    fast_slow = df.iloc[0]["Motoneuron"] # 8:resistance 9:prim/sec low/high rin need to see what is fast/slow
    key = (freq,signal_type,cell_type,fast_slow)
    waveforms[key] = abf_waveform



21.14896 21.18859
Empty DataFrame
Columns: [Time, Current]
Index: []
21.18859 21.21925
Empty DataFrame
Columns: [Time, Current]
Index: []
21.21925 21.24664
Empty DataFrame
Columns: [Time, Current]
Index: []
21.24664 21.28479
Empty DataFrame
Columns: [Time, Current]
Index: []
21.28479 21.328580000000002
Empty DataFrame
Columns: [Time, Current]
Index: []
21.328580000000002 21.36532
Empty DataFrame
Columns: [Time, Current]
Index: []
21.36532 21.40086
Empty DataFrame
Columns: [Time, Current]
Index: []
21.40086 22.32693
Empty DataFrame
Columns: [Time, Current]
Index: []
22.32693 22.35145
Empty DataFrame
Columns: [Time, Current]
Index: []
22.35145 22.37623
Empty DataFrame
Columns: [Time, Current]
Index: []
22.37623 22.400290000000002
Empty DataFrame
Columns: [Time, Current]
Index: []
22.400290000000002 22.42205
Empty DataFrame
Columns: [Time, Current]
Index: []
22.42205 22.42929
Empty DataFrame
Columns: [Time, Current]
Index: []
22.42929 22.44764
Empty DataFrame
Columns: [Time, Current]
Inde

/var/folders/mw/pz4gm95j6zb7syl48vm11wrw0000gn/T/ipykernel_17732/4113477500.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Seconds"] = df['On time']*0.001


In [19]:
waveforms

{(np.float64(25.233409033558843),
  'Excitatory',
  'CaP',
  'primary'): Empty DataFrame
 Columns: [Time, Current, Phase]
 Index: [],
 (np.float64(32.61578604044627),
  'Excitatory',
  'CaP',
  'primary'): Empty DataFrame
 Columns: [Time, Current, Phase]
 Index: [],
 (np.float64(36.50967506389131),
  'Excitatory',
  'CaP',
  'primary'): Empty DataFrame
 Columns: [Time, Current, Phase]
 Index: [],
 (np.float64(26.212319790300285),
  'Excitatory',
  'CaP',
  'primary'): Empty DataFrame
 Columns: [Time, Current, Phase]
 Index: [],
 (np.float64(22.836263987212853),
  'Excitatory',
  'CaP',
  'primary'): Empty DataFrame
 Columns: [Time, Current, Phase]
 Index: [],
 (np.float64(27.21829069134327),
  'Excitatory',
  'CaP',
  'primary'): Empty DataFrame
 Columns: [Time, Current, Phase]
 Index: [],
 (np.float64(28.137310073156204),
  'Excitatory',
  'CaP',
  'primary'): Empty DataFrame
 Columns: [Time, Current, Phase]
 Index: [],
 (np.float64(1.0798319781442016),
  'Excitatory',
  'CaP',
  'pri

In [38]:
waveforms2

{(np.float64(64.59948320413463),
  'Excitatory',
  'vRoP',
  'primary'):            Time     Current     Phase  Normalized Current
 169422  1.69422 -286.987274  0.000000            0.495059
 169423  1.69423 -285.644501  0.000646            0.500235
 169424  1.69424 -286.376923  0.001292            0.497412
 169425  1.69425 -287.231415  0.001938            0.494118
 169426  1.69426 -287.597626  0.002584            0.492706
 ...         ...         ...       ...                 ...
 170965  1.70965 -222.900375  0.996770            0.742118
 170966  1.70966 -224.121078  0.997416            0.737412
 170967  1.70967 -224.975571  0.998062            0.734118
 170968  1.70968 -226.562485  0.998708            0.728000
 170969  1.70969 -229.003891  0.999354            0.718588
 
 [1548 rows x 4 columns],
 (np.float64(36.927621861152154),
  'Excitatory',
  'vRoP',
  'primary'):            Time     Current         Phase  Normalized Current
 170970  1.70970 -231.567368  8.199579e-15            0.

dict_items([((np.float64(64.59948320413463), 'Excitatory', 'vRoP', 'primary'),            Time     Current     Phase
169422  1.69422 -286.987274  0.000000
169423  1.69423 -285.644501  0.000646
169424  1.69424 -286.376923  0.001292
169425  1.69425 -287.231415  0.001938
169426  1.69426 -287.597626  0.002584
...         ...         ...       ...
170965  1.70965 -222.900375  0.996770
170966  1.70966 -224.121078  0.997416
170967  1.70967 -224.975571  0.998062
170968  1.70968 -226.562485  0.998708
170969  1.70969 -229.003891  0.999354

[1548 rows x 3 columns]), ((np.float64(36.927621861152154), 'Excitatory', 'vRoP', 'primary'),            Time     Current         Phase
170970  1.70970 -231.567368  8.199579e-15
170971  1.70971 -233.764633  3.692762e-04
170972  1.70972 -235.595688  7.385524e-04
170973  1.70973 -236.572250  1.107829e-03
170974  1.70974 -236.816391  1.477105e-03
...         ...         ...           ...
173673  1.73673 -141.967758  9.981536e-01
173674  1.73674 -142.211899  9.985

In [34]:
print(waveforms.keys())
print(waveforms2.keys())

dict_keys([(np.float64(64.59948320413463), 'Excitatory', 'vRoP', 'primary'), (np.float64(36.927621861152154), 'Excitatory', 'vRoP', 'primary'), (np.float64(26.88894864210805), 'Excitatory', 'vRoP', 'primary'), (np.float64(1.098671705907558), 'Excitatory', 'vRoP', 'primary'), (np.float64(38.68471953578248), 'Excitatory', 'vRoP', 'primary'), (np.float64(38.314176245211364), 'Excitatory', 'vRoP', 'primary'), (np.float64(31.57562361856638), 'Excitatory', 'vRoP', 'primary'), (np.float64(27.746947835737842), 'Excitatory', 'vRoP', 'primary'), (np.float64(32.33107015842258), 'Excitatory', 'vRoP', 'primary'), (np.float64(28.192839018889114), 'Excitatory', 'vRoP', 'primary'), (np.float64(0.8342510094437214), 'Excitatory', 'vRoP', 'primary'), (np.float64(33.01419610432494), 'Excitatory', 'vRoP', 'primary'), (np.float64(31.979533098816624), 'Excitatory', 'vRoP', 'primary'), (np.float64(34.80682213713951), 'Excitatory', 'vRoP', 'primary'), (np.float64(27.925160569672837), 'Excitatory', 'vRoP', 'pri

In [ ]:
def make_waveforms(abf,df):
    '''
    Function that takes an abf file and a df of the annotations
    Returns a dictionary with waveforms labeled by their frequency
    '''
    abf.setSweep(sweepNumber=0, channel=0)
    x = abf.sweepX
    y = abf.sweepY
    abf_df = pd.DataFrame({
        'Time': x,
        'Current': y
    })
    print("abf_df head:", abf_df.head())  # Check if abf_df is populated correctly

    # df["On time"] = pd.to_numeric(df["On time"], errors="coerce")
    # df["Seconds"] = df['On time']*0.001
    waveforms = {}
    for i in range(len(df) - 1):
        t_0 = df.iloc[i,6] 
        t_f = df.iloc[i+1,6]

        # phase_0 = int(t_0 * 100000)
        # phase_1 = int(t_f * 100000)
        # abf_waveform = abf_df.loc[phase_0:phase_1].copy()
        abf_waveform = abf_df[(abf_df["Time"] >= t_0) & (abf_df["Time"] <= t_f)].copy()
        print(f"abf_waveform shape (i={i}):", abf_waveform.shape) 
        # print(abf_waveform)
        phase = (abf_waveform["Time"] - t_0) / (t_f - t_0)
        abf_waveform["Phase"] = phase
        
        # NORMALIZE CURRENT ALSO, test this:
        
        # y_max = max(abf_waveform["Current"])
        # y_min = min(abf_waveform["Current"])
        # normalized = (abf_waveform["Current"] - y_min) / (y_max - y_min)
        # abf_waveform["Normalized Current"] = normalized
        #=====

        # Dict keys
        freq = 1 / (t_f-t_0) # CHECK = df.loc[i + 1, "Freq"]
        cell_type = df.iloc[0,5]
        signal_type = df.iloc[0,7]
        fast_slow = df.iloc[0,9] # 8:resistance 9:prim/sec low/high rin need to see what is fast/slow
        key = (freq,cell_type,signal_type,fast_slow)
        waveforms[key] = abf_waveform

    return waveforms